In [1]:
# !pip install -U --quiet "sagemaker>=1.14.2"
# !pip install -U --quiet opencv-python tensorflow-hub

In [1]:
#!rm -r export/

rm: cannot remove ‘export/’: No such file or directory


In [2]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()


Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
import numpy as np
import os, argparse
import boto3
import pandas as pd
from sagemaker import get_execution_role

In [4]:
role = get_execution_role()

In [5]:
mapping = {'normal': 0, 'pneumonia': 1, 'COVID-19': 2}
inv_mapping = {0: 'normal', 1: 'pneumonia', 2: 'COVID-19'}

In [6]:
weightspath = "Best_weights/COVID-net-large"
metaname = 'model.meta_eval'
ckptname = 'model-2069'

In [7]:
# Note: This directory structure will need to be followed - see notes for the next section
model_version = '1'
export_dir = 'export/Servo/' + model_version

In [8]:
from tensorflow.python.saved_model import builder
from tensorflow.python.saved_model.signature_def_utils import predict_signature_def
from tensorflow.python.saved_model import tag_constants
from tensorflow.python.saved_model.utils import build_tensor_info

In [9]:
new_graph = tf.Graph()
with tf.Session(graph=new_graph) as sess:
    loader = tf.train.import_meta_graph(os.path.join(weightspath, metaname))
    loader.restore(sess, os.path.join(weightspath, ckptname))
    
    # Create prediction signature to be used by TensorFlow Serving Predict API
    graph = tf.get_default_graph()
    
#     Image Tensor
    image_tensor = graph.get_tensor_by_name("input_1:0")
#     image_tensor = tf.reshape(image_tensor, (1, 224, 224, 3))
#     image_tensor = tf.identity(image_tensor, name='x')
    
#     Prediction Tensor
    pred_tensor = graph.get_tensor_by_name("dense_3/Softmax:0")
#     pred_tensor = tf.reshape(pred_tensor, (1, 3))
    
    print(image_tensor)
    print(pred_tensor)
    signature = predict_signature_def(
        inputs={"inputs": image_tensor}, outputs={"scores": pred_tensor})
    # Export checkpoint to SavedModel
    builder = builder.SavedModelBuilder(export_dir)
    # Save the meta graph and variables
    builder.add_meta_graph_and_variables(
        sess=sess, tags=[tag_constants.SERVING], signature_def_map={"serving_default": signature})
    builder.save()

INFO:tensorflow:Restoring parameters from Best_weights/COVID-net-large/model-2069
Tensor("input_1:0", shape=(?, 224, 224, 3), dtype=float32)
Tensor("dense_3/Softmax:0", shape=(?, 3), dtype=float32)
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: export/Servo/1/saved_model.pb


In [12]:
# !saved_model_cli show --dir {export_dir} --all



MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['inputs'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 224, 224, 3)
        name: input_1:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['scores'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 3)
        name: dense_3/Softmax:0
  Method name is: tensorflow/serving/predict


In [10]:
!ls export/Servo/1
!ls export/Servo/1/variables/

saved_model.pb	variables
variables.data-00000-of-00001  variables.index


In [11]:
import tarfile
with tarfile.open('model.tar.gz', mode='w:gz') as archive:
    archive.add('export', recursive=True)

In [12]:
import sagemaker

In [13]:
sagemaker_session = sagemaker.Session()

In [14]:
inputs = sagemaker_session.upload_data(path='model.tar.gz', key_prefix='model')

In [15]:
!touch train.py

In [16]:
model_data = inputs or 's3://' + sagemaker_session.default_bucket() + '/model/model.tar.gz'

In [17]:
# from sagemaker.tensorflow.model import TensorFlowModel
from sagemaker.tensorflow.serving import Model

sagemaker_model = Model(model_data = model_data,
                        role = role,
                        framework_version='1.15.2',
                        entry_point = 'train.py')

In [ ]:
%%time
endpoint_name = 'covid19-endpoint'
predictor = sagemaker_model.deploy(initial_instance_count = 1, instance_type='ml.m4.xlarge', endpoint_name = endpoint_name)

-------------!CPU times: user 1min 15s, sys: 8.87 s, total: 1min 24s
Wall time: 8min 1s


In [19]:
endpoint_name = predictor.endpoint
endpoint_name

'covid19-endpoint'

In [20]:
from sagemaker.tensorflow.model import TensorFlowModel
predictor=sagemaker.tensorflow.model.TensorFlowPredictor(endpoint_name, sagemaker_session)

In [107]:
## Testing image:
imagepath = 'Data/all/sample_data/IMG10.jpeg'

In [116]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [117]:
x = cv2.imread(imagepath)
x = cv2.resize(x, (224, 224))
x = x.astype('float32') / 255.0

In [118]:
x.shape

(224, 224, 3)

In [119]:
x = np.expand_dims(x, axis=0)

In [26]:
# x

In [52]:
prediction = predictor.predict(x)

In [53]:
pred_list = prediction['predictions'][0]

In [54]:
max_index = pred_list.index(max(pred_list))

In [55]:
inv_mapping[max_index]

'normal'

In [130]:
#x

In [120]:
import json
import boto3
import numpy as np
import io
endpoint_name = 'covid19-endpoint'
client = boto3.client('runtime.sagemaker')
# The sample model expects an input of shape [1,50]
data = x
payload = json.dumps(data.tolist())
response = client.invoke_endpoint(EndpointName=endpoint_name,ContentType='application/json', Body=payload)
prediction = json.loads(response['Body'].read().decode())

In [121]:
prediction

{'predictions': [[0.689290226, 0.123658881, 0.187050864]]}

In [85]:
pred_list = prediction['predictions'][0]

In [86]:
max_index = pred_list.index(max(pred_list))

In [87]:
inv_mapping[max_index]

'normal'

In [88]:
import base64

In [122]:
s = base64.b64encode(x)

In [129]:
imgdata = base64.b64decode(s)
t = np.frombuffer(imgdata, np.float32)

In [132]:
t = np.reshape(t, (1, 224, 224, 3))